### Usando a implementação do modelo Word2Vec da biblioteca: gensim

In [ ]:
!pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import gensim as gensim
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.scripts import word2vec2tensor
from gensim.models.callbacks import CallbackAny2Vec
import re
from google.colab import drive

### Esse notebook Jupyter foi criado usando Google Colab. Essa linha apenas monta a sua pasta do drive para salvar os arquivos dos modelos gerados em uma pasta que você pode definir o nome nas linhas abaixo (TCCLG nesse caso)

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/My Drive/TCCLG

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TCCLG


### Essa parte do código carrega o csv das diferentes representações das palavras de um dicionário e faz um pequeno parsing, construindo também o conjunto de palavras que vão ser incorporadas nesse modelo

In [ ]:
df = pd.read_csv('etiquetas2.csv')
df = df.replace(':','', regex=True)
print(df.head())
restricted_word_set = df['palavra'].to_numpy()

      palavra  ... Unnamed: 4
0         aba  ...        NaN
1       ababa  ...        NaN
2      abaçai  ...        NaN
3      abaças  ...        NaN
4  abacateiro  ...        NaN

[5 rows x 5 columns]


### A função abaixo serve para restringir um "vocabulário" do word2vec. Nesse contexto, utilizamos isto para remover as silabas individuais geradas mais a frente para que o modelo retorne apenas palavras completas. Isso fica mais claro mais a frente.

In [ ]:
# AVISO: quando você cria o modelo pela primeira vez, a propriedade vectors_norm é igual a None.
# Essa propriedade só é atualizada com o valor dos vetores após o primeiro uso, no formato numpy.ndarray.
# Dessa forma, antes de usar a função abaixo, é necessário utilizar o modelo uma primeira vez para
# inicializar os valores. Basta chamar algo como most_similar("gato").
def restrict_w2v(w2v, restricted_word_set):
    new_vectors = []
    new_vocab = {}
    new_index2entity = []
    new_vectors_norm = []

    for i in range(len(w2v.wv.vocab)):
        word = w2v.wv.index2entity[i]
        vec = w2v.wv.vectors[i]
        vocab = w2v.wv.vocab[word]
        vec_norm = w2v.wv.vectors_norm[i]
        if word in restricted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)
            new_vectors_norm.append(vec_norm)

    w2v.wv.vocab = new_vocab
    w2v.wv.vectors = np.array(new_vectors)
    w2v.wv.index2entity = new_index2entity
    w2v.wv.index2word = new_index2entity
    w2v.wv.vectors_norm = new_vectors_norm

### Isso é apenas uma classe de logger para que consigamos ver o progresso do treinamento do modelo. Isso é um parâmetro da função, passado como callback

In [ ]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

        
epoch_logger = EpochLogger()

# Modelos utilizando representação fonética

---



### Modelo básico (Frases Simples):
#### Nesse modelo, cada "frase'' do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas. Exemplo: ["abacaxi", "a", "ba", "ca", "'xi"]

In [ ]:
df_fonetica = df[['palavra', 'fonetica']]
print(df_fonetica.head())
print("")

      palavra         fonetica
0         aba            'a.bə
1       ababa         a.'ba.bə
2      abaçai       a.ba.sa.'i
3      abaças        a.'ba.səs
4  abacateiro  a.ba.ka.'tey.ɾʊ



#### O começo da "frase" é a palavra original e as "silabas" são concatenadas na frase para que similaridades possam ser encontradas em um nível silábico pelo Word2Vec. A ideia aqui é que palavras com silabas semelhantes devem ter um alto nível de similaridade pelo modelo. Como as silabas aparecendo "proximas" das palavras na frase, essa semelhança é identificada até certo ponto pelo Word2Vec.

In [ ]:
corpus_fonetica_basico = []
for index, row in df_fonetica.iterrows():
  sentence = [row['palavra']]
  for syllable in row['fonetica'].split('.'):
    sentence.append(syllable)
  corpus_fonetica_basico.append(sentence)

In [ ]:
for k, v in enumerate(corpus_fonetica_basico[:2]):
  print(f'{k}: {v}')

0: ['aba', "'a", 'bə']
1: ['ababa', 'a', "'ba", 'bə']


#### Após montar a representação, treinamos o modelo com os parâmetros definidos abaixo após multiplos testes. O modelo foi treinado por 30 iterações.

In [ ]:
model_fonetica_basico = Word2Vec(corpus_fonetica_basico, min_count=1, workers=8, window=10, sg=1, size=300, iter=30, callbacks=[epoch_logger])

# AVISO: esse é o print de primeiro uso mencionado anteriormente para inicializar os valores necessários para a função restrict_w2v
print(model_fonetica_basico.wv.most_similar('abaçai'))

Epoch #30 start
Epoch #30 end
Epoch #31 start
Epoch #31 end
Epoch #32 start
Epoch #32 end
Epoch #33 start
Epoch #33 end
Epoch #34 start
Epoch #34 end
Epoch #35 start
Epoch #35 end
Epoch #36 start
Epoch #36 end
Epoch #37 start
Epoch #37 end
Epoch #38 start
Epoch #38 end
Epoch #39 start
Epoch #39 end
Epoch #40 start
Epoch #40 end
Epoch #41 start
Epoch #41 end
Epoch #42 start
Epoch #42 end
Epoch #43 start
Epoch #43 end
Epoch #44 start
Epoch #44 end
Epoch #45 start
Epoch #45 end
Epoch #46 start
Epoch #46 end
Epoch #47 start
Epoch #47 end
Epoch #48 start
Epoch #48 end
Epoch #49 start
Epoch #49 end
Epoch #50 start
Epoch #50 end
Epoch #51 start
Epoch #51 end
Epoch #52 start
Epoch #52 end
Epoch #53 start
Epoch #53 end
Epoch #54 start
Epoch #54 end
Epoch #55 start
Epoch #55 end
Epoch #56 start
Epoch #56 end
Epoch #57 start
Epoch #57 end
Epoch #58 start
Epoch #58 end
Epoch #59 start
Epoch #59 end
[('imbassai', 0.9387959241867065), ('sai', 0.9182076454162598), ('abaiba', 0.8913350105285645), ('sa

#### Por fim, restringimos o vocabulário do modelo para remover as silabas. Dessa forma, as semelhanças encontradas são apenas com as palavras completas das frases geradas acima.

In [ ]:
restrict_w2v(model_fonetica_basico, restricted_word_set)

#### Alguns exemplos dos resultados obtidos de coeficientes de similaridades para diferentes palavras. É facil notar a similaridade de algumas destas:

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_fonetica_basico.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_fonetica_basico.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_fonetica_basico.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
capita: 0.9363455772399902
apitando: 0.9342569708824158
piata: 0.9284365177154541
decapitando: 0.91983962059021
capitana: 0.9089586734771729

Most similar to gue:
gui: 0.9705873727798462
guido: 0.9572112560272217
guia: 0.9545613527297974
guizar: 0.9470541477203369
guida: 0.9376347661018372

Most similar to grisalha:
grisalhos: 0.9421676993370056
grisalho: 0.9412848949432373
bizarras: 0.883394718170166
gozaram: 0.8716270923614502
gozarem: 0.871281623840332


#### Essa linha salva o modelo na pasta local. Caso esteja usando o colab, salvará na pasta definida nas primeiras linhas desse documento.

In [ ]:
model_fonetica_basico.wv.save_word2vec_format("model_fonetica_basico.model", binary=True)

#### Essa linha salva o arquivo do formato tensorflow que pode ser utilizado com o modelo acima em https://projector.tensorflow.org/ para uma visualização dos dados em forma de gráfico 3D com clusters de palavras similares.

In [ ]:
word2vec2tensor.word2vec2tensor("model_fonetica_basico.model", "tensor-model-fonetica-basico", True)

### Modelo Contagem (Frases com contador relativo a silaba)
#### Nesse modelo, cada frase do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas, com silabas duplicadas sendo diferenciadas por um contador utilizado como prefixo. Essa estratégia garante que o modelo não seja influenciado no aprendizado a acreditar que palavras que possuem silabas similares duplicadas possuem um grau de similaridade maior. Exemplo: ["anarcocomunismo", "0-a", "0-nar", "0-co", "1-co", "0-mu", "0-'nis", "0-mo"]


In [ ]:
corpus_fonetica_contagem = []
for index, row in df_fonetica.iterrows():
  sentence = [row['palavra']]
  syllables = []
  for syllable in row['fonetica'].split('.'):
    modified_syllable = f'{syllables.count(syllable)}-{syllable}'
    syllables.append(syllable)
    sentence.append(modified_syllable)
  corpus_fonetica_contagem.append(sentence)

In [ ]:
for k, v in enumerate(corpus_fonetica_contagem[:3]):
  print(f'{k}: {v}')

0: ['aba', "0-'a", '0-bə']
1: ['ababa', '0-a', "0-'ba", '0-bə']
2: ['abaçai', '0-a', '0-ba', '0-sa', "0-'i"]


In [ ]:
model_fonetica_contagem = Word2Vec(corpus_fonetica_contagem, min_count=1, workers=8, window=30, sg=1, size=300, iter=30, callbacks=[epoch_logger])
print(model_fonetica_contagem.wv.most_similar('abaçai'))

Epoch #60 start
Epoch #60 end
Epoch #61 start
Epoch #61 end
Epoch #62 start
Epoch #62 end
Epoch #63 start
Epoch #63 end
Epoch #64 start
Epoch #64 end
Epoch #65 start
Epoch #65 end
Epoch #66 start
Epoch #66 end
Epoch #67 start
Epoch #67 end
Epoch #68 start
Epoch #68 end
Epoch #69 start
Epoch #69 end
Epoch #70 start
Epoch #70 end
Epoch #71 start
Epoch #71 end
Epoch #72 start
Epoch #72 end
Epoch #73 start
Epoch #73 end
Epoch #74 start
Epoch #74 end
Epoch #75 start
Epoch #75 end
Epoch #76 start
Epoch #76 end
Epoch #77 start
Epoch #77 end
Epoch #78 start
Epoch #78 end
Epoch #79 start
Epoch #79 end
Epoch #80 start
Epoch #80 end
Epoch #81 start
Epoch #81 end
Epoch #82 start
Epoch #82 end
Epoch #83 start
Epoch #83 end
Epoch #84 start
Epoch #84 end
Epoch #85 start
Epoch #85 end
Epoch #86 start
Epoch #86 end
Epoch #87 start
Epoch #87 end
Epoch #88 start
Epoch #88 end
Epoch #89 start
Epoch #89 end
[('imbassai', 0.9414029121398926), ('sai', 0.906258225440979), ('saraui', 0.8930704593658447), ('bad

In [ ]:
restrict_w2v(model_fonetica_contagem, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_fonetica_contagem.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_fonetica_contagem.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_fonetica_contagem.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_fonetica_contagem.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
capita: 0.9319568872451782
apitando: 0.9248915314674377
decapitando: 0.9182816743850708
piata: 0.9141545295715332
capitana: 0.9016276597976685

Most similar to gue:
gui: 0.9686412811279297
guizar: 0.9460459351539612
guida: 0.9459826946258545
erguido: 0.9262365102767944
guido: 0.924612820148468

Most similar to grisalha:
grisalhos: 0.9427147507667542
grisalho: 0.940769612789154
gozarem: 0.9019870162010193
gozaram: 0.9000802040100098
usaram: 0.8853440284729004

Most similar to sintetizado:
sintetizados: 0.9815362691879272
sintetizada: 0.9810668230056763
sintetizadas: 0.9785659313201904
sinterizado: 0.9617933630943298
sintetizaram: 0.9604758024215698


In [ ]:
model_fonetica_contagem.wv.save_word2vec_format("model_fonetica_contagem.model", binary=True)

In [ ]:
word2vec2tensor.word2vec2tensor("model_fonetica_contagem.model", "tensor-model-fonetica-contagem", True)

### Modelo Chunks (frases com sub-palavras)
#### Nesse modelo, cada frase do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas. Além disso, são geradas todas sub-palavras possíveis com as silabas levando em consideração a ordem das mesmas na palavra original. Essa estratégia tem como objetivo tentar influenciar o aprendizado a detectar similaridade de contexto com base em sub-palavras que podem ocorrer dentro de palavras maiores, similarmente a estrategia do FastText mas considerando silabas ao invés de n-grams. Exemplo: ["abacaxi", "a", "ba", "ca", "'xi", "aba", "baca", "ca'xi", "abaca", "baca'xi", "abaca'xi"]


In [ ]:
corpus_fonetica_chunk = []
for index, row in df_fonetica.iterrows():
  sentence = [row['palavra']]
  syllables = []
  modified_syllables = []
  chunks = []
  for syllable in row['fonetica'].split('.'):
    modified_syllable = f'{syllable}'
    syllables.append(syllable)
    modified_syllables.append(modified_syllable)
    sentence.append(modified_syllable)
  for i in range(2, len(modified_syllables)+1):
    for j in range(0, len(modified_syllables)):
      start = j
      end = j + i
      if (end <= len(modified_syllables)):
        chunk = ' '.join(syllables[start:end])
        sentence.append(f"{chunk}")
        chunks.append(chunk)
  corpus_fonetica_chunk.append(sentence)

In [ ]:
for k, v in enumerate(corpus_fonetica_chunk[:3]):
  print(f'{k}: {v}')

0: ['aba', "'a", 'bə', "'a bə"]
1: ['ababa', 'a', "'ba", 'bə', "a 'ba", "'ba bə", "a 'ba bə"]
2: ['abaçai', 'a', 'ba', 'sa', "'i", 'a ba', 'ba sa', "sa 'i", 'a ba sa', "ba sa 'i", "a ba sa 'i"]


In [ ]:
model_fonetica_chunk = Word2Vec(corpus_fonetica_chunk, min_count=1, workers=8, window=50, sg=1, size=300, iter=30, callbacks=[epoch_logger])
print(model_fonetica_chunk.wv.most_similar('abaçai'))

Epoch #90 start
Epoch #90 end
Epoch #91 start
Epoch #91 end
Epoch #92 start
Epoch #92 end
Epoch #93 start
Epoch #93 end
Epoch #94 start
Epoch #94 end
Epoch #95 start
Epoch #95 end
Epoch #96 start
Epoch #96 end
Epoch #97 start
Epoch #97 end
Epoch #98 start
Epoch #98 end
Epoch #99 start
Epoch #99 end
Epoch #100 start
Epoch #100 end
Epoch #101 start
Epoch #101 end
Epoch #102 start
Epoch #102 end
Epoch #103 start
Epoch #103 end
Epoch #104 start
Epoch #104 end
Epoch #105 start
Epoch #105 end
Epoch #106 start
Epoch #106 end
Epoch #107 start
Epoch #107 end
Epoch #108 start
Epoch #108 end
Epoch #109 start
Epoch #109 end
Epoch #110 start
Epoch #110 end
Epoch #111 start
Epoch #111 end
Epoch #112 start
Epoch #112 end
Epoch #113 start
Epoch #113 end
Epoch #114 start
Epoch #114 end
Epoch #115 start
Epoch #115 end
Epoch #116 start
Epoch #116 end
Epoch #117 start
Epoch #117 end
Epoch #118 start
Epoch #118 end
Epoch #119 start
Epoch #119 end
[('a ba sa', 0.9884077906608582), ("a ba sa 'i", 0.987059056

In [ ]:
restrict_w2v(model_fonetica_chunk, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_fonetica_chunk.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_fonetica_chunk.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_fonetica_chunk.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_fonetica_chunk.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to aba:')
for k, v in enumerate(model_fonetica_chunk.wv.most_similar('aba')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bata:')
for k, v in enumerate(model_fonetica_chunk.wv.most_similar('bata')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bantu:')
for k, v in enumerate(model_fonetica_chunk.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
ipitanga: 0.9072718620300293
itapitanga: 0.8963677883148193
pitanga: 0.8764810562133789
ibirapitanga: 0.8739174604415894
capita: 0.8720220923423767

Most similar to gue:
gui: 0.9817230105400085
guigui: 0.9255589246749878
guia: 0.9249275326728821
bagui: 0.9219368696212769
guizar: 0.9208035469055176

Most similar to grisalha:
grisalho: 0.9255660772323608
grisalhos: 0.9249950647354126
pisaram: 0.7750778198242188
pisavam: 0.7743526101112366
pisarem: 0.7676078081130981

Most similar to sintetizado:
sintetizada: 0.9636496305465698
sintetizando: 0.9626451134681702
sintetizadas: 0.961909294128418
sintetizados: 0.9585564732551575
sintetizou: 0.9539825916290283

Most similar to aba:
goiaba: 0.9426910877227783
piaba: 0.9364608526229858
diaba: 0.9029510021209717
acaiaba: 0.8832888603210449
caioaba: 0.8697571754455566

Most similar to bata:
batava: 0.931865394115448
batalham: 0.9131442904472351
batalha: 0.8922882080078125
batavo: 0.8886007070541382
batatas: 0.883752107620

In [ ]:
model_fonetica_chunk.wv.save_word2vec_format("model_fonetica_chunk.model", binary=True)

In [ ]:
word2vec2tensor.word2vec2tensor("model_fonetica_chunk.model", "tensor-fonetica-chunk", True)

### Contagem + Chunks (frases com sub-palavras e contador)
#### Nesse modelo, cada frase do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas,  com silabas duplicadas sendo diferenciadas por um contador utilizado como prefixo. Além disso, são geradas todas sub-palavras possíveis com as silabas levando em consideração a ordem das mesmas na palavra original. As sub-palavras também possuem um prefixo de contador de número de ocorrências. O objetivo é combinar as duas estratégias anteriores e obter as vantagens de ambas estratégias. Exemplo: ["abacaxi", "0-a", "0-ba", "0-ca", "0-'xi", "0\_aba", "0\_baca", "0\_ca'xi", "0\_abaca", "0\_baca'xi", "0\_abaca'xi"]

In [ ]:
corpus_fonetica_contagem_chunk = []
for index, row in df_fonetica.iterrows():
  sentence = [row['palavra']]
  syllables = []
  modified_syllables = []
  chunks = []
  for syllable in row['fonetica'].split('.'):
    modified_syllable = f'{syllables.count(syllable)}-{syllable}'
    syllables.append(syllable)
    modified_syllables.append(modified_syllable)
    sentence.append(modified_syllable)
  for i in range(2, len(modified_syllables)+1):
    for j in range(0, len(modified_syllables)):
      start = j
      end = j + i
      if (end <= len(modified_syllables)):
        chunk = ' '.join(syllables[start:end])
        sentence.append(f"{chunks.count(chunk)}_{chunk}")
        chunks.append(chunk)
  corpus_fonetica_contagem_chunk.append(sentence)

In [ ]:
for k, v in enumerate(corpus_fonetica_contagem_chunk[:3]):
  print(f'{k}: {v}')

0: ['aba', "0-'a", '0-bə', "0_'a bə"]
1: ['ababa', '0-a', "0-'ba", '0-bə', "0_a 'ba", "0_'ba bə", "0_a 'ba bə"]
2: ['abaçai', '0-a', '0-ba', '0-sa', "0-'i", '0_a ba', '0_ba sa', "0_sa 'i", '0_a ba sa', "0_ba sa 'i", "0_a ba sa 'i"]


In [ ]:
model_fonetica_contagem_chunk = Word2Vec(corpus_fonetica_contagem_chunk, min_count=1, workers=8, window=50, sg=1, size=300, iter=30, callbacks=[epoch_logger])
print(model_fonetica_contagem_chunk.wv.most_similar('abaçai'))

Epoch #120 start
Epoch #120 end
Epoch #121 start
Epoch #121 end
Epoch #122 start
Epoch #122 end
Epoch #123 start
Epoch #123 end
Epoch #124 start
Epoch #124 end
Epoch #125 start
Epoch #125 end
Epoch #126 start
Epoch #126 end
Epoch #127 start
Epoch #127 end
Epoch #128 start
Epoch #128 end
Epoch #129 start
Epoch #129 end
Epoch #130 start
Epoch #130 end
Epoch #131 start
Epoch #131 end
Epoch #132 start
Epoch #132 end
Epoch #133 start
Epoch #133 end
Epoch #134 start
Epoch #134 end
Epoch #135 start
Epoch #135 end
Epoch #136 start
Epoch #136 end
Epoch #137 start
Epoch #137 end
Epoch #138 start
Epoch #138 end
Epoch #139 start
Epoch #139 end
Epoch #140 start
Epoch #140 end
Epoch #141 start
Epoch #141 end
Epoch #142 start
Epoch #142 end
Epoch #143 start
Epoch #143 end
Epoch #144 start
Epoch #144 end
Epoch #145 start
Epoch #145 end
Epoch #146 start
Epoch #146 end
Epoch #147 start
Epoch #147 end
Epoch #148 start
Epoch #148 end
Epoch #149 start
Epoch #149 end
[("0_a ba sa 'i", 0.9871894717216492), (

In [ ]:
restrict_w2v(model_fonetica_contagem_chunk, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_fonetica_contagem_chunk.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_fonetica_contagem_chunk.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_fonetica_contagem_chunk.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_fonetica_contagem_chunk.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to aba:')
for k, v in enumerate(model_fonetica_contagem_chunk.wv.most_similar('aba')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bata:')
for k, v in enumerate(model_fonetica_contagem_chunk.wv.most_similar('bata')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bantu:')
for k, v in enumerate(model_fonetica_contagem_chunk.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
ipitanga: 0.9192812442779541
itapitanga: 0.9060196876525879
pitanga: 0.8937732577323914
ibipitanga: 0.8884360790252686
ibirapitanga: 0.8807494640350342

Most similar to gue:
gui: 0.9711884260177612
bagui: 0.9276586174964905
segui: 0.9209465980529785
guizar: 0.9205312728881836
guigui: 0.914503812789917

Most similar to grisalha:
grisalho: 0.9310243725776672
grisalhos: 0.926767885684967
gozarem: 0.7934274673461914
gozavam: 0.7734292149543762
gozaram: 0.7691957950592041

Most similar to sintetizado:
sintetizada: 0.9674462080001831
sintetizadas: 0.9666929244995117
sintetizaram: 0.9646207094192505
sintetizados: 0.9634042978286743
sintetizou: 0.959270715713501

Most similar to aba:
piaba: 0.9431962370872498
goiaba: 0.9404555559158325
diaba: 0.9093843698501587
acaiaba: 0.8863286972045898
guaraciaba: 0.8818168044090271

Most similar to bata:
batava: 0.9217967987060547
batalha: 0.9026615023612976
batalham: 0.8982866406440735
batatas: 0.8926267027854919
batalhas: 0.890

In [ ]:
model_fonetica_contagem_chunk.wv.save_word2vec_format("model_fonetica_contagem_chunk.model", binary=True)

In [ ]:
word2vec2tensor.word2vec2tensor("model_fonetica_contagem_chunk.model", "tensor-fonetica-contagem-chunk", True)

### FastText - Contagem + Chunks
#### Esse modelo acabou sendo discutido apenas brevemente no TCC, mas é a mesma representação acima sendo utilizada para treinar outro modelo, o FastText, que diferentemente do Word2Vec, tem por padrão a funcionalidade de identificar similaridades em silabas sem necessidade da inclusão delas na "frase". Apesar disso, a detecção de silabas do FastText é implementada pela utilização de n-grams, que são pequenas parcelas da palavra de tamanhos variados. Isso não é bem preciso para as nossas representações de etiquetagem e de fonética, visto que existem símbolos que quando removidos tornam completamente diferente a representação da palavra. Apesar disso, foi decidido tentar testar os resultados obtidos com esse outro modelo para comparação com o Word2Vec.

In [ ]:
corpus_fonetica_fast_text_cc = []
for index, row in df_fonetica.iterrows():
  sentence = [row['palavra']]
  syllables = []
  modified_syllables = []
  chunks = []
  for syllable in row['fonetica'].split('.'):
    modified_syllable = f'{syllables.count(syllable)}-{syllable}'
    syllables.append(syllable)
    modified_syllables.append(modified_syllable)
    sentence.append(modified_syllable)
  for i in range(2, len(modified_syllables)+1):
    for j in range(0, len(modified_syllables)):
      start = j
      end = j + i
      if (end <= len(modified_syllables)):
        chunk = ' '.join(syllables[start:end])
        sentence.append(f"{chunks.count(chunk)}_{chunk}")
        chunks.append(chunk)
  corpus_fonetica_fast_text_cc.append(sentence)

In [ ]:
for k, v in enumerate(corpus_fonetica_fast_text_cc[:3]):
  print(f'{k}: {v}')

0: ['aba', "0-'a", '0-bə', "0_'a bə"]
1: ['ababa', '0-a', "0-'ba", '0-bə', "0_a 'ba", "0_'ba bə", "0_a 'ba bə"]
2: ['abaçai', '0-a', '0-ba', '0-sa', "0-'i", '0_a ba', '0_ba sa', "0_sa 'i", '0_a ba sa', "0_ba sa 'i", "0_a ba sa 'i"]


In [ ]:
model_fonetica_fast_text_cc = FastText(corpus_fonetica_fast_text_cc, min_count=1, workers=8, window=10, sg=1, size=300, iter=30, min_n=2, max_n=2, callbacks=[epoch_logger])
print(model_fonetica_fast_text_cc.wv.most_similar('simtetizando'))

Epoch #150 start
Epoch #150 end
Epoch #151 start
Epoch #151 end
Epoch #152 start
Epoch #152 end
Epoch #153 start
Epoch #153 end
Epoch #154 start
Epoch #154 end
Epoch #155 start
Epoch #155 end
Epoch #156 start
Epoch #156 end
Epoch #157 start
Epoch #157 end
Epoch #158 start
Epoch #158 end
Epoch #159 start
Epoch #159 end
Epoch #160 start
Epoch #160 end
Epoch #161 start
Epoch #161 end
Epoch #162 start
Epoch #162 end
Epoch #163 start
Epoch #163 end
Epoch #164 start
Epoch #164 end
Epoch #165 start
Epoch #165 end
Epoch #166 start
Epoch #166 end
Epoch #167 start
Epoch #167 end
Epoch #168 start
Epoch #168 end
Epoch #169 start
Epoch #169 end
Epoch #170 start
Epoch #170 end
Epoch #171 start
Epoch #171 end
Epoch #172 start
Epoch #172 end
Epoch #173 start
Epoch #173 end
Epoch #174 start
Epoch #174 end
Epoch #175 start
Epoch #175 end
Epoch #176 start
Epoch #176 end
Epoch #177 start
Epoch #177 end
Epoch #178 start
Epoch #178 end
Epoch #179 start
Epoch #179 end
[('sintetizando', 0.9414820671081543), (

In [ ]:
restrict_w2v(model_fonetica_fast_text_cc, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_fonetica_fast_text_cc.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_fonetica_fast_text_cc.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_fonetica_fast_text_cc.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_fonetica_fast_text_cc.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to aba:')
for k, v in enumerate(model_fonetica_fast_text_cc.wv.most_similar('aba')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bata:')
for k, v in enumerate(model_fonetica_fast_text_cc.wv.most_similar('bata')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bantu:')
for k, v in enumerate(model_fonetica_fast_text_cc.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
apita: 0.8644018173217773
tagua: 0.8421937227249146
piata: 0.8282225131988525
ipigua: 0.8035833835601807
capiata: 0.8031173944473267

Most similar to gue:
guegue: 0.9656793475151062
ague: 0.919357419013977
fugue: 0.8675894141197205
guel: 0.8402413129806519
guegues: 0.8358191251754761

Most similar to grisalha:
grisalho: 0.9335529804229736
grisalhos: 0.8917711973190308
isaurinha: 0.7998642325401306
isauria: 0.7702336311340332
brisas: 0.7620376944541931

Most similar to sintetizado:
sintetizados: 0.9423842430114746
sintetizando: 0.9279612898826599
sintetizador: 0.9259282350540161
sintetiza: 0.9171642065048218
sintetizadores: 0.9060614705085754

Most similar to aba:
ababa: 0.9645570516586304
abaiba: 0.9523106813430786
abacha: 0.9137357473373413
baba: 0.9116197824478149
jaba: 0.9013693928718567

Most similar to bata:
taba: 0.9488698840141296
baeta: 0.9247950315475464
bacata: 0.9064058065414429
tatajuba: 0.9037512540817261
batalha: 0.9024170637130737

Most similar

In [ ]:
print(model_fonetica_fast_text_cc.wv.most_similar('simtetizando'))
print(model_fonetica_fast_text_cc.wv.most_similar('facista'))
print(model_fonetica_fast_text_cc.wv.most_similar('facistas'))
print(model_fonetica_fast_text_cc.wv.most_similar('panda'))
print(model_fonetica_fast_text_cc.wv.most_similar('pamda'))
print(model_fonetica_fast_text_cc.wv.most_similar('caxorro'))
print(model_fonetica_fast_text_cc.wv.most_similar('bantu'))

[('sintetizando', 0.9414820671081543), ('simpatizando', 0.863528847694397), ('sintetizado', 0.848122239112854), ('sintetiza', 0.8260803818702698), ('sistematizando', 0.8209267854690552), ('simpatizante', 0.8127805590629578), ('sizandro', 0.796578049659729), ('tematizando', 0.7934931516647339), ('sintetizados', 0.7923383712768555), ('profetizando', 0.790247917175293)]
[('facistas', 0.8731343746185303), ('cista', 0.8502378463745117), ('sista', 0.8319255113601685), ('neofascista', 0.8129968047142029), ('antifascista', 0.7979865670204163), ('consista', 0.7733446359634399), ('acis', 0.7705721259117126), ('musicista', 0.7683302164077759), ('cenecista', 0.7670172452926636), ('fascistas', 0.7610845565795898)]
[('fascistas', 0.9639461040496826), ('cistas', 0.9066466689109802), ('fantasistas', 0.9049100875854492), ('neofascistas', 0.889751672744751), ('antifascistas', 0.8703123331069946), ('fadistas', 0.8637509942054749), ('neofascista', 0.8630149364471436), ('anti-fascistas', 0.8594833612442017

In [ ]:
model_fonetica_fast_text_cc.wv.save_word2vec_format("model_fonetica_fast_text_cc.model", binary=False)
word2vec2tensor.word2vec2tensor("model_fonetica_fast_text_cc.model", "tensor-model-fonetica-fast-text-cc", False)